In [2]:
from fairworkflows import FairStep, FairWorkflow, mark_as_fairstep

## Use the `mark_as_fairstep` decorator to mark a function as a FAIR step

It triggers validation:

In [5]:
@mark_as_fairstep()
def add(a: int, b: int) -> int:
    """
    Computational step adding two ints together.
    """
    return a + b

AssertionError: Step RDF has no rdfs:label
Step RDF must be either a bpmn:ManualTask or a bpmn:ScriptTask


In [36]:
@mark_as_fairstep(label='add integers', is_script_task=True)
def add(a: int, b: int) -> int:
    """
    Computational step adding two ints together.
    """
    return a + b

In [37]:
@mark_as_fairstep(label='square root', is_script_task=True)
def sqrt(a: int) -> float:
    """
    Computational step taking the square root of an integer.
    """
    return a ** 0.5

## Define the workflow

In [51]:
workflow = FairWorkflow(description='A simple workflow', label='test workflow')
step1 = FairStep.from_function(add)
step2 = FairStep.from_function(sqrt)
workflow.add(step1)
workflow.add(step2, follows=step1)

## Publish using the API
one could use the notebook extenstion, it doesn't have the newest features yet though

In [52]:
step1.publish_as_nanopub(use_test_server=True)

Published to http://purl.org/np/RAcQHJ_Nxq6cUQkzg8J-xOiBeRee5aLQXf_fAebFwQDks
Published concept to http://purl.org/np/RAcQHJ_Nxq6cUQkzg8J-xOiBeRee5aLQXf_fAebFwQDks#step


{'nanopub_uri': 'http://purl.org/np/RAcQHJ_Nxq6cUQkzg8J-xOiBeRee5aLQXf_fAebFwQDks',
 'concept_uri': 'http://purl.org/np/RAcQHJ_Nxq6cUQkzg8J-xOiBeRee5aLQXf_fAebFwQDks#step'}

## Execution of steps
Execution of steps is just done by calling the functions

In [58]:
intermediate_result = add(1, 3)
sqrt(intermediate_result)

2.0

You can inject raw code from steps fetched from nanopub:

In [56]:
@mark_as_fairstep(label='add integers', is_script_task=True)
def add(a: int, b: int) -> int:
    """
    Computational step adding two ints together.
    """
    return a + b

And just call the function:

In [57]:
result = add(1, 3)